In [4]:
from google.cloud import storage
from google.oauth2 import service_account
import os
import pandas as pd
from datetime import datetime, timedelta

In [37]:
#load in last three days from gcs

forecast_day = datetime.today() #'20200310'
period = 3 #days

def get_time_dates(forecast_day, period):
    end = datetime.today()
    start = datetime.today() + timedelta(-period)
    delta = end-start

    date_list = [(start + timedelta(i)).strftime('%Y%m%d') for i in range(delta.days+1)]

    time_pairs = list()

    for i in range(delta.days+1):
        begin_time = (start + timedelta(i-1)).strftime('%Y%m%d')
        begin_time = f'{begin_time}T2300'
        end_time = (start + timedelta(i)).strftime('%Y%m%d')
        end_time = f'{end_time}T2300'

        time_pairs.append((begin_time, end_time))

    print(time_pairs)
    
    return time_pairs



    
get_time_dates(forecast_day, period)

[('20200306T2300', '20200307T2300'), ('20200307T2300', '20200308T2300'), ('20200308T2300', '20200309T2300')]


[('20200306T2300', '20200307T2300'),
 ('20200307T2300', '20200308T2300'),
 ('20200308T2300', '20200309T2300')]

In [56]:
#load in the date range of data
creds = service_account.Credentials.from_service_account_file(os.environ['GOOGLE_APPLICATION_CREDENTIALS'])
project_id = 'ml-energy-dashboard'
bucket='ml-energy-dashboard-raw-data'
folder_name = 'raw-days'

client = storage.Client(credentials=creds, project=project_id)
bucket = client.get_bucket(bucket)

time_pairs = get_time_dates(forecast_day, period)

data_list = list()

for time_pair in time_pairs:
    file_name = f'es-energy-demand-{time_pair[0]}-{time_pair[1]}'
    print(f'Downloading ... {file_name}')
    blob = bucket.blob(f'{folder_name}/{file_name}')
    data_json=blob.download_as_string()
    data = pd.read_json(data_json, typ='series', orient='records', keep_default_dates=False)
    data_list.append(data)

[('20200306T2300', '20200307T2300'), ('20200307T2300', '20200308T2300'), ('20200308T2300', '20200309T2300')]


In [60]:
data = pd.concat(data_list, axis=0)
data.index = data.index.tz_localize('UTC').tz_convert('Europe/Madrid')
data

2020-03-07 00:00:00+01:00    26752
2020-03-07 01:00:00+01:00    25060
2020-03-07 02:00:00+01:00    23886
2020-03-07 03:00:00+01:00    23240
2020-03-07 04:00:00+01:00    22860
2020-03-07 05:00:00+01:00    22855
2020-03-07 06:00:00+01:00    23405
2020-03-07 07:00:00+01:00    24483
2020-03-07 08:00:00+01:00    26202
2020-03-07 09:00:00+01:00    28719
2020-03-07 10:00:00+01:00    30237
2020-03-07 11:00:00+01:00    30435
2020-03-07 12:00:00+01:00    29937
2020-03-07 13:00:00+01:00    29876
2020-03-07 14:00:00+01:00    29133
2020-03-07 15:00:00+01:00    27540
2020-03-07 16:00:00+01:00    26378
2020-03-07 17:00:00+01:00    26009
2020-03-07 18:00:00+01:00    26348
2020-03-07 19:00:00+01:00    28490
2020-03-07 20:00:00+01:00    30129
2020-03-07 21:00:00+01:00    30139
2020-03-07 22:00:00+01:00    28725
2020-03-07 23:00:00+01:00    26836
2020-03-08 00:00:00+01:00    24985
2020-03-08 01:00:00+01:00    23210
2020-03-08 02:00:00+01:00    21964
2020-03-08 03:00:00+01:00    21315
2020-03-08 04:00:00+

In [64]:
def persistance(series):
    date = (datetime.today()+timedelta(-1)).strftime('%Y%m%d')
    return series[date]

In [147]:
def persistance_day_ma(series, num_days, date):
    
    window=24*num_days
    rolling_mean = series.rolling(window=window, min_periods=24, closed='right').mean()
    
    return rolling_mean[date]

In [145]:
def persistance_MA_hourly(series, days):
    
    df = series.groupby(series.index.hour).mean()
    
    return df
    

In [67]:
p1 = persistance(data)

In [68]:
p2 = persistance_day_ma(data, 3)

In [126]:
data

2020-03-07 00:00:00+01:00    26752
2020-03-07 01:00:00+01:00    25060
2020-03-07 02:00:00+01:00    23886
2020-03-07 03:00:00+01:00    23240
2020-03-07 04:00:00+01:00    22860
2020-03-07 05:00:00+01:00    22855
2020-03-07 06:00:00+01:00    23405
2020-03-07 07:00:00+01:00    24483
2020-03-07 08:00:00+01:00    26202
2020-03-07 09:00:00+01:00    28719
2020-03-07 10:00:00+01:00    30237
2020-03-07 11:00:00+01:00    30435
2020-03-07 12:00:00+01:00    29937
2020-03-07 13:00:00+01:00    29876
2020-03-07 14:00:00+01:00    29133
2020-03-07 15:00:00+01:00    27540
2020-03-07 16:00:00+01:00    26378
2020-03-07 17:00:00+01:00    26009
2020-03-07 18:00:00+01:00    26348
2020-03-07 19:00:00+01:00    28490
2020-03-07 20:00:00+01:00    30129
2020-03-07 21:00:00+01:00    30139
2020-03-07 22:00:00+01:00    28725
2020-03-07 23:00:00+01:00    26836
2020-03-08 00:00:00+01:00    24985
2020-03-08 01:00:00+01:00    23210
2020-03-08 02:00:00+01:00    21964
2020-03-08 03:00:00+01:00    21315
2020-03-08 04:00:00+

In [146]:
p3 = persistance_MA_hourly(data, 3, datetime.today().strftime('%Y%m%d'))
p3

0     25431.000000
1     23780.000000
2     22613.000000
3     22056.333333
4     21898.333333
5     22225.000000
6     23574.666667
7     25285.666667
8     27010.000000
9     28878.000000
10    30067.000000
11    30228.333333
12    29947.000000
13    29871.333333
14    29139.666667
15    27991.000000
16    27109.000000
17    26860.333333
18    27104.000000
19    29537.666667
20    31288.000000
21    31003.666667
22    29279.333333
23    27040.333333
dtype: float64

In [88]:
persist_forecast = pd.concat([p1, p2, p3], axis=1)
persist_forecast.columns = ['naive', 'MA3-day', 'MA30day-hbh']

In [89]:
date = datetime.today().strftime('%Y%m%d')
persist_forecast.index = pd.DatetimeIndex(pd.date_range(start=f'{date}T0000', end=f'{date}T2300', freq='H'))
persist_forecast

,naive,MA3-day,MA30day-hbh
2020-03-10 00:00:00,24556,25940.816327,25431.000000
2020-03-10 01:00:00,23070,25883.400000,23780.000000
2020-03-10 02:00:00,21989,25807.039216,22613.000000
2020-03-10 03:00:00,21614,25726.403846,22056.333333
2020-03-10 04:00:00,21757,25651.509434,21898.333333
2020-03-10 05:00:00,22605,25595.092593,22225.000000
2020-03-10 06:00:00,25764,25598.163636,23574.666667
2020-03-10 07:00:00,29636,25670.267857,25285.666667
2020-03-10 08:00:00,32126,25783.526316,27010.000000
2020-03-10 09:00:00,33268,25912.568966,28878.000000


In [172]:
"""
Google Cloud Function

Triggered by a google cloud cron daily at 0110

Accesses the raw predictions in GCS and calcualtes the persistance forecast for the next day's benchmark predictions.

Types of persistance forecasts available:

1. Persistance: Previous day's demand used as today's forecast.
2. Persistance 3 Day Moving Average: Applies a moving average to the last 3 days.
3. Persistance Hourly-by-hour 3 day moving average: Applies a moving average for each hour of the day for the last 3 days.

"""


def gen_persistance_forecasts(request):
    from datetime import datetime, timedelta
    import pandas as pd
    import numpy as np
    from google.cloud import storage
    
    FOLDER_DOWN = 'raw-days'
    FOLDER_UP = 'persistance_forecasts'
    BUCKET = 'ml-energy-dashboard-raw-data'
    
    def get_time_dates(period):
        end = datetime.today()
        start = datetime.today() + timedelta(-period)
        delta = end-start

        time_pairs = list()

        for i in range(delta.days+1):
            begin_time = (start + timedelta(i-1)).strftime('%Y%m%d')
            begin_time = f'{begin_time}T2300'
            end_time = (start + timedelta(i)).strftime('%Y%m%d')
            end_time = f'{end_time}T2300'

            time_pairs.append((begin_time, end_time))
        
        return time_pairs 
    
    def raw_data_date():
        return (datetime.today()+timedelta(-1)).strftime('%Y%m%d')
    
    def gcs_save_name(date):
        return f'es-persistance-forecasts-{date}'
    
    def gcs_load_name(start, end):
        return f'es-energy-demand-{start}-{end}'

    def get_gcs_data(client, bucket_name, folder_name, file_name):
        
        bucket = client.get_bucket(bucket_name)
        blob = bucket.blob(f'{folder_name}/{file_name}')
        data_json = blob.download_as_string()
        
        return pd.read_json(data_json, typ='series', orient='records', keep_default_dates=False)
    
    def upload_data_to_gcs(client, data, date, bucket_name, folder_name, file_name):
        
        bucket = client.get_bucket(bucket_name)
        blob = bucket.blob(f'{folder_name}/{file_name}')
        blob.upload_from_string(data.to_json())
    
    def reset_data_index(data_list):

        data = pd.concat(data_list, axis=0)
        data.index = data.index.tz_localize('UTC').tz_convert('Europe/Madrid')

        return data


    def persistance(series, date):
        return series[date]

    def persistance_day_ma(series, num_days, date):
    
        window=24*num_days
        rolling_mean = series.rolling(window=window, min_periods=24, closed='right').mean()
    
        return rolling_mean[date]

    def persistance_MA_hourly(series, days):
        
        df = series.groupby(series.index.hour).mean()
    
        return df

    def calc_persistance_forecasts(data):
        date = (datetime.today()+timedelta(-1)).strftime('%Y%m%d')
        
        p1 = persistance(data, date)
        p2 = persistance_day_ma(data, 3, date)
        p3 = persistance_MA_hourly(data, 3)
        
        data = np.vstack([p1.values, p2.values, p3.values]).T
        persist_df = pd.DataFrame(data, columns=['naive', 'MA3-day', 'MA30day-hbh'])

        today = datetime.today().strftime('%Y%m%d')
        persist_df.index = pd.DatetimeIndex(pd.date_range(start=f'{today}T0000', end=f'{today}T2300', freq='H'))
        
        return persist_df

    params = request#.get_json()
    
    if 'gen_persist' in params and params['gen_persist']:
        #storage_client = storage.Client()

        creds = service_account.Credentials.from_service_account_file(os.environ['GOOGLE_APPLICATION_CREDENTIALS'])
        storage_client = storage.Client(credentials=creds, project=project_id)
        #download the data to make the persistance forecast
        time_pairs = get_time_dates(3)

        data_list = list()

        for time_pair in time_pairs:

            file_name = f'es-energy-demand-{time_pair[0]}-{time_pair[1]}'
            data = get_gcs_data(storage_client, BUCKET, FOLDER_DOWN, file_name)
            data_list.append(data)
            
        data = reset_data_index(data_list)

        #calcuate the persistance forecasts
        persistance_forecasts = calc_persistance_forecasts(data)

    return persistance_forecasts
        #upload to gcs
        #persistance_file_name = gcs_save_name(datetime.today())
        #create persistance - simple persistance today's values >> tomorrow forecast
        #upload_data_to_gcs(storage_client, data, date_to_forecast, BUCKET, FOLDER_UP, file_to_forecast)


In [173]:
request = {"gen_persist": 'true'}
persist = gen_persistance_forecasts(request)
persist

(2020-03-10 00:00:00+01:00    26025
 2020-03-10 01:00:00+01:00    24464
 2020-03-10 02:00:00+01:00    23602
 2020-03-10 03:00:00+01:00    22989
 2020-03-10 04:00:00+01:00    23066
 2020-03-10 05:00:00+01:00    23649
 2020-03-10 06:00:00+01:00    26358
 2020-03-10 07:00:00+01:00    29863
 2020-03-10 08:00:00+01:00    32001
 2020-03-10 09:00:00+01:00    32997
 2020-03-10 10:00:00+01:00    33063
 2020-03-10 12:00:00+01:00    32666
 2020-03-10 13:00:00+01:00    32583
 2020-03-10 14:00:00+01:00    31553
 2020-03-10 15:00:00+01:00    30862
 2020-03-10 16:00:00+01:00    30254
 2020-03-10 17:00:00+01:00    29920
 2020-03-10 18:00:00+01:00    29832
 2020-03-10 19:00:00+01:00    32071
 2020-03-10 20:00:00+01:00    33985
 2020-03-10 21:00:00+01:00    33242
 2020-03-10 22:00:00+01:00    30704
 2020-03-10 23:00:00+01:00    27988
 dtype: int64,
 2020-03-10 00:00:00+01:00    27061.367347
 2020-03-10 01:00:00+01:00    27009.420000
 2020-03-10 02:00:00+01:00    26942.607843
 2020-03-10 03:00:00+01:00  

In [176]:
len(persist[0]),len(persist[1]),len(persist[2]) 

(23, 23, 24)

Date 20200310 has a missing value coming from the API. Will need to develop a function that interpolates this value by mapping a new index.